train.csv: card_ids + information about the card itself + target
test.csv: card_ids + information about the card itself

historical_transaction: designed to join with train.csv, test.csv, merchants.csv
new_merchant_transaction.csv: same

merchants

ideas:
1. Join train, transaction, and merchants into a table.
2. XGboost regression.

In [28]:
import pandas as pd
import numpy as np

In [29]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
# hist_trans = pd.read_csv('historical_transactions.csv')
hist_trans = pd.read_csv('small_historical_transactions.csv')
new_trans = pd.read_csv('new_merchant_transactions.csv')
merchants = pd.read_csv('merchants.csv')

In [30]:
hist_trans.shape

(29112361, 14)

In [31]:
# combine hist_trans and new_trans
# trans = 

# join train and hist_trans
# card_trans = pd.merge(train, hist_trans, how='inner', on='card_id')
# card_trans.to_csv('card_trans.csv')
# card_trans = pd.read_csv('card_trans.csv')

In [32]:
# card_trans.head()

In [33]:
# print(card_trans['card_id'].unique().shape)
# print(card_trans.shape)


In [ ]:
def generate_small_hist():
    hist_trans = pd.read_csv('historical_transactions.csv')    
    # get small parts of data occuring over the past two months
    hist_trans['purchase_date'] = pd.to_datetime(hist_trans['purchase_date'])
    # hist_trans['purchase_date'].describe()
    small_hist = hist_trans[(hist_trans['purchase_date'] > '2018-01-01') & (hist_trans['purchase_date'] < '2018-03-01')]
    small_hist.to_csv('small_historical_transactions_20180101_20180301.csv')
    small_hist.dtypes

In [34]:


# testing snippets

# test = hist_trans.head()
# test['purchase_date'] = pd.to_datetime(test['purchase_date'])
# test

# # mask = test['purchase_date'] > '2017-6-1' & test['purchase_date'] < '2017-9-1'
# mask = (test['purchase_date'] > '2017-06-01') & (test['purchase_date'] < '2017-09-01')
# test_2 = test[mask]
# test_3 = test_2['purchase_date']
# test_3.describe()
# # test_2.describe(include='all')

authorized_flag                 object
card_id                         object
city_id                          int64
category_1                      object
installments                     int64
category_3                      object
merchant_category_id             int64
merchant_id                     object
month_lag                        int64
purchase_amount                float64
purchase_date           datetime64[ns]
category_2                     float64
state_id                         int64
subsector_id                     int64
dtype: object

In [5]:
train_label = card_trans['target']
train_data = card_trans.drop('target', axis=1)

In [6]:
train_data.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,Unnamed: 0,authorized_flag,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,2017-06,C_ID_92a2005557,5,2,1,12454878,Y,69,N,0,A,560,M_ID_b3c49066d8,-1,-0.665614,2018-01-27 06:44:29,1.0,9,34
1,2017-06,C_ID_92a2005557,5,2,1,12454882,Y,69,N,0,A,560,M_ID_b3c49066d8,0,-0.595230,2018-02-25 09:25:39,1.0,9,34
2,2017-06,C_ID_92a2005557,5,2,1,12454898,Y,69,N,0,A,454,M_ID_6b718b517f,-1,-0.712347,2018-01-09 10:06:54,1.0,9,39
3,2017-06,C_ID_92a2005557,5,2,1,12454899,Y,69,N,0,A,879,M_ID_00a6ca8a8a,0,-0.707899,2018-02-17 09:39:17,1.0,9,29
4,2017-06,C_ID_92a2005557,5,2,1,12454901,Y,69,N,0,A,195,M_ID_cc72dac895,-1,-0.589580,2018-01-13 14:23:54,1.0,9,34


delete unmeaningful columns

In [7]:
#remove ids

ids = ['card_id', 'subsector_id', 'city_id', 'merchant_category_id', 'merchant_id', 'state_id', 'subsector_id', 'subsector_id']

train_data = train_data.drop(ids, axis=1)

In [8]:
cat_col = train_data.dtypes[train_data.dtypes == 'object'].index.values
cat_col

array(['first_active_month', 'authorized_flag', 'category_1',
       'category_3', 'purchase_date'], dtype=object)

missing values

In [9]:
missing_values = train_data.isna().sum() / train_data.shape[0]
missing_values

first_active_month    0.000000
feature_1             0.000000
feature_2             0.000000
feature_3             0.000000
Unnamed: 0            0.000000
authorized_flag       0.000000
category_1            0.000000
installments          0.000000
category_3            0.014265
month_lag             0.000000
purchase_amount       0.000000
purchase_date         0.000000
category_2            0.106382
dtype: float64

In [ ]:
# study the distribution of columns containing missing values

# small_train['category_3'].hist()

In [10]:
train_data.shape

(3273086, 13)

In [ ]:
# from sklearn.impute import SimpleImputer

# imputer = SimpleImputer(strategy='most_frequent')
# train_data = pd.DataFrame(imputer.fit_transform(train_data), columns = train_data.columns)
# train_data.isna().sum() / train_data.shape[0]

In [11]:
def encode_feature(data, columns):
    # encode each feature
    for col in columns:
        data[col] = data[col].astype('category')
        data[col] = data[col].cat.codes    
        
encode_feature(train_data, cat_col)

In [12]:
train_data.head()
# train_data.columns

,first_active_month,feature_1,feature_2,feature_3,Unnamed: 0,authorized_flag,category_1,installments,category_3,month_lag,purchase_amount,purchase_date,category_2
0,65,5,2,1,12454878,1,0,0,0,-1,-0.665614,960144,1.0
1,65,5,2,1,12454882,1,0,0,0,0,-0.595230,2003829,1.0
2,65,5,2,1,12454898,1,0,0,0,-1,-0.712347,281542,1.0
3,65,5,2,1,12454899,1,0,0,0,0,-0.707899,1707635,1.0
4,65,5,2,1,12454901,1,0,0,0,-1,-0.589580,449232,1.0


In [13]:
import xgboost as xgb

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [14]:
def grid_eval(model, train_data, train_label, params):
    grid = GridSearchCV(cls, param_grid=params, cv=5, scoring='neg_mean_squared_error')
    grid_res = grid.fit(train_data, train_label)

    print('best parameters: ', grid_res.best_params_)
    print('best results: ', np.sqrt(np.mean(-cross_val_score(grid_res.best_estimator_, train_data, train_label, cv=5, scoring='neg_mean_squared_error'))))
    
def cv_eval(model, train_data, train_label):
    print('best results: ', np.sqrt(np.mean(-cross_val_score(model, train_data, train_label, cv=5, scoring='neg_mean_squared_error'))))

In [15]:
params = [{"n_estimators":[100]}]

cls = xgb.XGBRegressor(n_estimators=100)

# grid = GridSearchCV(cls, param_grid=params, cv=5, scoring='neg_mean_squared_error')
# grid_res = grid.fit(train_data, train_label)

# print('best parameters: ', grid_res.best_params_)
# print('best results: ', np.sqrt(np.mean(-cross_val_score(grid_res.best_estimator_, train_data, train_label, cv=5, scoring='neg_mean_squared_error'))))

cv_eval(cls, train_data, train_label)

best results:  1.9495646850817872


In [1]:
%%time

print('a')

a
CPU times: user 366 µs, sys: 209 µs, total: 575 µs
Wall time: 306 µs
